In [5]:
import seaborn as sns
import numpy as np 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt 
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D
from tensorflow.keras.optimizers import Adam 
from keras_tuner import RandomSearch
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import pandas as pd
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import warnings
warnings.filterwarnings("ignore")


In [8]:
# count no of images in respective classes 0- Brain tumor 1-Healthy
ROOT_DIR="brain-tumor-classification-mri"
number_of_images = {}

for dir in os.listdir(ROOT_DIR):
    number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))

In [9]:
number_of_images.items()

dict_items([('Testing', 4), ('Training', 4)])

In [11]:
len(os.listdir("brain-tumor-classification-mri"))

2

## ALREADY THE DATA IS SPLITED FOR TRAINING AND TESTING
## 70% TRAINING
## 15% TESTING
## 15%TESTING


## Model Building


In [2]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras

In [3]:
# CNN Model

model= Sequential()
model.add(Conv2D(filters= 16, kernel_size= (3,3), activation= 'relu', input_shape = (224,224,3), padding= 'same')) #First Layer of the model

model.add(Conv2D(filters= 36, kernel_size= (3,3), activation= 'relu',)) #Second Layer of the model
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters= 64, kernel_size= (3,3), activation= 'relu',)) #Layer of the model
model.add(MaxPool2D(pool_size=(2,2))) #third layer of the model

model.add(Conv2D(filters= 128, kernel_size= (3,3), activation= 'relu',)) #Layer of the model
model.add(MaxPool2D(pool_size=(2,2))) #Fourth layer of the model

model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1, activation= 'sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 36)      5220      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 36)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 109, 109, 64)      20800     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 52, 52, 128)       7

In [6]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
def preprocessingImages1(path):

    image_data = ImageDataGenerator(zoom_range=0.2, shear_range= 0.2, rescale=1/255, horizontal_flip= True)
    image= image_data.flow_from_directory(directory = path, target_size = (224,224), batch_size= 32, class_mode = 'binary')

    return image

In [19]:
path= 'brain-tumor-classification-mri\Training'
train_data = preprocessingImages1(path)

Found 2870 images belonging to 4 classes.


In [20]:
def preprocessingImages2(path):

    image_data = ImageDataGenerator(rescale=1/255)
    image= image_data.flow_from_directory(directory = path, target_size = (224,224), batch_size= 32, class_mode = 'binary')

    return image

In [29]:
path = 'brain-tumor-classification-mri\Testing'
test_data= preprocessingImages2(path)

Found 394 images belonging to 4 classes.


In [32]:
path = 'brain-tumor-classification-mri\val'
val_data= preprocessingImages2(path)

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'brain-tumor-classification-mri\x0bal'

## Early stopping

In [24]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

#Early Stopping
es= EarlyStopping(monitor="val_accuracy", min_delta= 0.01, patience = 3, verbose = 1,mode='auto')

#Model CheckPoint
mc= ModelCheckpoint(monitor="val_accuracy", filepath="./bestmodel.h5",verbose =1, save_best_only= True, mode='auto')

cd = [es,mc]

In [ ]:
hs = model.fit_generator(generator = train_data, steps_per_epoch= 8, epochs= 30, verbose=1, validation_data